In [2]:
import numpy as np
import xgboost as xgb
import pandas as pd
import category_encoders as ce
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
import csv

In [58]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = train_data

train_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)

labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
train_data['sub_grade'] = labelencoder.fit_transform(train_data['sub_grade'])
train_data['home_ownership'] = labelencoder.fit_transform(train_data['home_ownership'])
train_data['emp_length'].replace('< 1 year', 0.5, inplace=True)
train_data['emp_length'].replace('1 year', 1.0, inplace=True)
train_data['emp_length'].replace('2 years', 2.0, inplace=True)
train_data['emp_length'].replace('3 years', 3.0, inplace=True)
train_data['emp_length'].replace('4 years', 4.0, inplace=True)
train_data['emp_length'].replace('5 years', 5.0, inplace=True)
train_data['emp_length'].replace('6 years', 6.0, inplace=True)
train_data['emp_length'].replace('7 years', 7.0, inplace=True)
train_data['emp_length'].replace('8 years', 8.0, inplace=True)
train_data['emp_length'].replace('9 years', 9.0, inplace=True)
train_data['emp_length'].replace('10 years', 10.0, inplace=True)
train_data['emp_length'].replace('10+ years', 15.0, inplace=True)
train_data['emp_length'] = train_data['emp_length'].fillna(0)

train_data['mort_acc'] = train_data['mort_acc'].fillna(0)
# Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]

X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_train.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0.0, inplace=True)
y_train_numeric.replace('Charged Off', 1.0, inplace=True)



In [60]:
# X_val_numeric = X_train_numeric.sample(frac=0.2)
# val_indices = X_val_numeric.index
# X_train_numeric = X_train_numeric.drop(val_indices)
# train_indices = X_train_numeric.index
# Y_val = y_train_numeric.drop(train_indices)
# y_train = y_train_numeric.drop(val_indices)

# print(train_indices)
# print(val_indices)

train_X, val_X, train_y, val_y = sklearn.model_selection.train_test_split(X_train_numeric, y_train_numeric, test_size=0.2, random_state=0)

        loan_amnt  term_(months)  int_rate  installment  sub_grade  \
89545        9800             36     19.52       361.82         18   
190443      16000             60     18.49       410.58         16   
57541       13275             36      6.62       407.60          1   
124679       3000             36     16.29       105.91         13   
119027       5000             36     12.35       166.91          8   
...           ...            ...       ...          ...        ...   
152315      24175             36     12.12       804.35          7   
176963      35000             36     15.80      1227.05         12   
117952       2400             36     14.33        82.42         11   
173685       7750             36     12.12       257.86          7   
43567       12175             36     10.99       398.54          6   

        emp_length  home_ownership  annual_inc    dti  open_acc  pub_rec  \
89545          5.0               4     43372.8  26.28        14        0   
190443 

In [63]:
model = xgb.XGBClassifier(max_depth=6, min_child_weight=1,  n_estimators=30,
                            n_jobs=-1,learning_rate=0.2, alpha=10)
model.fit(train_X, train_y)
xgb_predictions = model.predict_proba(val_X)[:,1]
print('AUC XGB')
print(sklearn.metrics.roc_auc_score(val_y, xgb_predictions))
print(sklearn.metrics.roc_auc_score(train_y, model.predict_proba(train_X)[:,1]))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:24:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUC XGB
0.6848892678484035
0.7105160979433875


END OF XGBOOST TRAINING, START OF CNN

In [ ]:
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torch.nn as nn

In [ ]:
train_data = pd.read_csv('./data/LOANS_TRAIN.csv')
test_data = pd.read_csv('./data/LOANS_TEST.csv')

id_column = test_data['id']

train_data.drop(columns=['id','grade', 'emp_title', 'title', 'earliest_cr_line', 'issue_d', 'zip_code'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title', 'earliest_cr_line', 'issue_d', 'zip_code'], axis=1, inplace=True)

# we want to ultimately use this data, but its nominal multi-categorical nature requires further preprocessing
# prolly with OneHotEncoder
    # Deal with nominal, multi-categorical data
    # Goal: convert home_ownership (RENT, OWN, MORTGAGE, OTHER) to a 4D vector.
        # if MORTGAGE ==> [0,0,1,0]
# print(len(train_data['zip_code'].unique()))
# print(train_data['zip_code'].unique())
ce_OHE = ce.OneHotEncoder(cols=['addr_state', 'home_ownership', 'purpose'])
print('BEFORE')
print(train_data.shape)
train_data = ce_OHE.fit_transform(train_data)
print('AFTER')
print(train_data.shape)
test_data = ce_OHE.fit_transform(test_data)
# NOTE: 'purpose' only has 14 categories, probably good things to learn from

# there are no joint applications in training, so model won't be able to learn it. Better to drop it
train_data.drop(columns=['application_type', 'purpose_14'], axis=1, inplace=True)
test_data.drop(columns=['application_type'], axis=1, inplace=True)


labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
# Assigning numerical values and storing in another column
train_data['sub_grade'] = labelencoder.fit_transform(train_data['sub_grade'])
train_data['emp_length'].replace('< 1 year', 0, inplace=True)
train_data['emp_length'].replace('1 year', 1.0, inplace=True)
train_data['emp_length'].replace('2 years', 2.0, inplace=True)
train_data['emp_length'].replace('3 years', 3.0, inplace=True)
train_data['emp_length'].replace('4 years', 4.0, inplace=True)
train_data['emp_length'].replace('5 years', 5.0, inplace=True)
train_data['emp_length'].replace('6 years', 6.0, inplace=True)
train_data['emp_length'].replace('7 years', 7.0, inplace=True)
train_data['emp_length'].replace('8 years', 8.0, inplace=True)
train_data['emp_length'].replace('9 years', 9.0, inplace=True)
train_data['emp_length'].replace('10 years', 10.0, inplace=True)
train_data['emp_length'].replace('10+ years', 15.0, inplace=True)
train_data['emp_length'] = train_data['emp_length'].fillna(0)
train_data['pub_rec_bankruptcies'] = train_data['pub_rec_bankruptcies'].fillna(0)
train_data['verification_status'].replace('Verified', 1, inplace=True)
train_data['verification_status'].replace('Source Verified', 1, inplace=True)
train_data['verification_status'].replace('Not Verified', 0, inplace=True)
train_data['initial_list_status'].replace('w', 1, inplace=True)
train_data['initial_list_status'].replace('f', 0, inplace=True)

test_data['sub_grade'] = labelencoder2.fit_transform(test_data['sub_grade'])
test_data['emp_length'].replace('< 1 year', 0, inplace=True)
test_data['emp_length'].replace('1 year', 1.0, inplace=True)
test_data['emp_length'].replace('2 years', 2.0, inplace=True)
test_data['emp_length'].replace('3 years', 3.0, inplace=True)
test_data['emp_length'].replace('4 years', 4.0, inplace=True)
test_data['emp_length'].replace('5 years', 5.0, inplace=True)
test_data['emp_length'].replace('6 years', 6.0, inplace=True)
test_data['emp_length'].replace('7 years', 7.0, inplace=True)
test_data['emp_length'].replace('8 years', 8.0, inplace=True)
test_data['emp_length'].replace('9 years', 9.0, inplace=True)
test_data['emp_length'].replace('10 years', 10.0, inplace=True)
test_data['emp_length'].replace('10+ years', 15.0, inplace=True)
test_data['emp_length'] = test_data['emp_length'].fillna(0)
test_data['pub_rec_bankruptcies'] = test_data['pub_rec_bankruptcies'].fillna(0)
test_data['verification_status'].replace('Verified', 1, inplace=True)
# technically encoding two categories into one... might want to change
test_data['verification_status'].replace('Verified', 1, inplace=True)
test_data['verification_status'].replace('Source Verified', 1, inplace=True)
test_data['verification_status'].replace('Not Verified', 0, inplace=True)
test_data['initial_list_status'].replace('w', 1, inplace=True)
test_data['initial_list_status'].replace('f', 0, inplace=True)

train_data['mort_acc'] = train_data['mort_acc'].fillna(0)

test_data['mort_acc'] = test_data['mort_acc'].fillna(0)
# Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)
train_data['revol_util'] = train_data['revol_util'].fillna(0)
test_data['revol_util'] = test_data['revol_util'].fillna(0)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]
X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_test.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0.0, inplace=True)
y_train_numeric.replace('Charged Off', 1.0, inplace=True)


In [ ]:
X_val_numeric = X_train_numeric.drop(train_indices)
Y_val = y_train.drop(train_indices)
X_train_numeric = X_train_numeric.drop(val_indices)
y_train = y_train.drop(val_indices)

In [ ]:
for col in X_train_numeric.columns:
    if X_train_numeric[col].isnull().values.any() > 0:
        print(col)
    x_min = X_train_numeric[col].min()
    x_max = X_train_numeric[col].max()
    X_train_numeric[col] = (X_train_numeric[col] - x_min) / (x_max - x_min)
    # also need to normalize test data, but must use training data
    X_test_numeric[col] = (X_test_numeric[col] - x_min) / (x_max - x_min)
    X_test_numeric[col].mask(X_test_numeric[col] < 0, 0, inplace=True)
    X_test_numeric[col].mask(X_test_numeric[col] > 1, 1, inplace=True)
    # TODO: DETERMINE WHAT TO DO IN THESE CASES!!!
    if X_test_numeric[col].min() < 0:
        print(f"min is less than zero in test data in column {col}")
        print(X_test_numeric[col].min())
    if X_test_numeric[col].max() > 1:
        print(f"max is more than one in test data in column {col}")
        print(X_test_numeric[col].max())

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(1, 3), stride=1, padding=0),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=(1, 2), stride=1),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1, 3), stride=1, padding=0),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=(1, 2), stride=1),
    nn.Flatten(),
    nn.Linear(5056, 512),
    nn.ReLU(),
    nn.Dropout(p=0.25),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

In [ ]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# Custom Dataset
from torch.utils.data import Dataset
import torch


class CustomDataset(Dataset):
    def __init__(self, X, Y):
        """
        X: a dataframe with rows as training examples and columns features
        Y: a pandas series with labels"""
        self.x_train=torch.tensor(X.values, dtype=torch.float32)
        self.y_train=torch.tensor(Y.values, dtype=torch.float32)
        self.x_train = torch.reshape(self.x_train, (self.x_train.shape[0], 1, 1, self.x_train.shape[1],))

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        return (self.x_train[idx], self.y_train[idx])

In [ ]:
from torch.utils.data import DataLoader, random_split

BATCH_SIZE = 512

ds_train = CustomDataset(X_train_numeric, y_train_numeric)
ds_val = CustomDataset(X_val_numeric, Y_val)
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
val_eval_loader = DataLoader(ds_val, batch_size=len_val, shuffle=False)

In [ ]:
n_epochs = 10  # paper used 100 originally
lr = 0.001

train_loss_hist = np.zeros([n_epochs, 1])
val_loss_hist = np.zeros([n_epochs, 1])
for epoch in range(n_epochs):
    print(f'EPOCH {epoch}')
    model.train()
    train_loss = 0
    for i, data in enumerate(train_loader):
        features, labels = data
        # print(type(features))
        # print(f"PESKY BASTARDS: {features.isnan().sum()}")
        optimizer.zero_grad()
        # forward pass
        output = model(features)
        output = torch.reshape(output, (output.shape[0], ))
        if output.isnan().sum() > 0:
            print(f"RIP{output.isnan().sum()}")
        output = torch.nan_to_num(output)
        # calculate loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        # update
        optimizer.step()
        # track training loss
        train_loss_hist[epoch] += loss.item()
    train_loss_hist[epoch] /= len(train_loader)
    print(f"\ttraining loss: {train_loss_hist[epoch]}")

    # validate
    with torch.no_grad():
        model.eval()
        output = None
        labels = None
        for i, data in enumerate(val_eval_loader):
            features, labels = data
            output = model(features)
            output = torch.reshape(output, (output.shape[0], ))
            output = torch.nan_to_num(output)
            loss = criterion(output, labels)
            val_loss_hist[epoch] += loss.item()
        preds = output.cpu().detach().numpy()
        ground_truths = labels.cpu().detach().numpy()
        print(f'\tAUC: {sklearn.metrics.roc_auc_score(ground_truths, preds)}')
        val_loss_hist[epoch] /= len(val_eval_loader)
        print(f"\tvalidation loss: {val_loss_hist[epoch]}")

In [ ]:
# validate
with torch.no_grad():
    model.eval()
    output = None
    labels = None
    for i, data in enumerate(val_eval_loader):
        features, labels = data
        output = model(features)
        output = torch.reshape(output, (output.shape[0], ))
        output = torch.nan_to_num(output)
        loss = criterion(output, labels)
        val_loss_hist[epoch] += loss.item()
    preds = output.cpu().detach().numpy()
    ground_truths = labels.cpu().detach().numpy()
    print(f'\tAUC: {sklearn.metrics.roc_auc_score(ground_truths, preds)}')
    val_loss_hist[epoch] /= len(val_eval_loader)
    print(f"\tvalidation loss: {val_loss_hist[epoch]}")

In [ ]:
# TODO: create points of the form (1, xgb pred, cnn pred) and plot them by color according to their ground truth label
# see if they are linearly separable

In [ ]:
# # Test Dataset

# class TestDataset(Dataset):
#     def __init__(self, X):
#         """
#         X: a dataframe with rows as training examples and columns features
#         Y: a pandas series with labels"""
#         self.x_train=torch.tensor(X.values, dtype=torch.float32)
#         self.x_train = torch.reshape(self.x_train, (self.x_train.shape[0], 1, 1, self.x_train.shape[1],))

#     def __len__(self):
#         print(f'GETTING LENGTH {self.x_train.shape[0]}')
#         return self.x_train.shape[0]

#     def __getitem__(self,idx):
#         return self.x_train[idx]

In [ ]:
# ds_test = TestDataset(X_test_numeric)

# test_loader = torch.utils.data.Dataloader(ds_test, batch_size=1, shuffle=True)

# with torch.no_grad():
#     model.eval()
#     cnn_predictions = []
#     for i, data in enumerate(test_loader):
#         features = data
#         output = model(features)
#         output = torch.reshape(output, (output.shape[0], ))
#         if output.isnan().sum() > 0:
#             print(f"RIP{output.isnan().sum()}")
#         output = torch.nan_to_num(output)
#         print(f'PREDICTION: {output.item()}')
#         cnn_predictions.append(output.item())



In [ ]:
ensemble_features = []

assert len(cnn_predictions) == len(xgb_predictions)

for i in range(len(cnn_predictions)):
    
